<table style="width:100%">
  <tr>
    <td><center style="font-size:300%;">Exemplo Prático (Emojify)</center></td>
    <td><img src="https://logodownload.org/wp-content/uploads/2015/02/puc-rio-logo.gif" width="100"/></td> 
  </tr>    
</table>

Msc. Cristian Muñoz V.

In [0]:
%%bash

rm -rf glove.6B.50d.zip glove.6B.50d.txt emo_utils.py train_emoji.csv test.csv
wget -O glove.6B.50d.zip https://github.com/crismunoz/DeepLearningExamples/raw/master/week2/resources/glove.6B.50d.zip
wget -O emo_utils.py https://raw.githubusercontent.com/crismunoz/DeepLearningExamples/master/week2/emo_utils.py
wget -O train_emoji.csv https://raw.githubusercontent.com/crismunoz/DeepLearningExamples/master/week2/train_emoji.csv
wget -O test.csv https://raw.githubusercontent.com/crismunoz/DeepLearningExamples/master/week2/test.csv
unzip glove.6B.50d.zip
pip install emoji

In [0]:
import numpy as np
from emo_utils import *
import emoji
import matplotlib.pyplot as plt

%matplotlib inline

In [0]:
X_train, Y_train = read_csv('train_emoji.csv')
X_test, Y_test = read_csv('test.csv')

In [0]:
X_train[1], Y_train[1]

In [0]:
maxLen = len(max(X_train, key=len).split())
print(maxLen)

In [0]:
index = 10
print(X_train[index], label_to_emoji(Y_train[index]))

In [0]:
Y_oh_train = convert_to_one_hot(Y_train, C = 5)
Y_oh_test = convert_to_one_hot(Y_test, C = 5)

In [0]:
index = 40
print(Y_train[index], "agora vai ser em one-hot encoding:", Y_oh_train[index])

In [0]:
file_name='glove.6B.50d.txt'
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs(file_name)

In [0]:
word = "homem"
index = 89846
print("O índice de", word, "no vocabulario de palavras é: ", word_to_index[word])
print("A palavra no index ", str(index) + " do vocabulario é: '{}'".format(index_to_word[index]))
print (word_to_vec_map[word])

# Definição do Modelo

In [0]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

In [0]:
def sentences_to_indices(X, word_to_index, max_len):   
    total_examples = X.shape[0]
    X_indices = np.zeros((len(X), max_len))
    
    for i in range(total_examples):
        sentence_words = X[i].lower().split()
        j = 0
        for w in sentence_words:
            X_indices[i, j] = word_to_index[w]
            j = j + 1
    
    return X_indices
  
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    
    vocab_len = len(word_to_index)+1                  # keras+1

    emb_dim = word_to_vec_map["cucumber"].shape[0]      # Word embedding
    
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False)

    embedding_layer.build((None,))
    
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer
  
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)

In [0]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)

# Modelo Simple

In [0]:
# Definir o modelo "model"
import keras.backend as K
from keras.layers import Lambda

# y = camada()(X) onde X=entrada
inp = Input(shape=(maxLen,))
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
embedding = embedding_layer(inp)
avg = Lambda(lambda x:K.mean(x, axis=1))(embedding)   # K chama o backend, no caso Tensorflow
logits = Dense(5)(avg)
output = Activation('softmax')(logits)

model = Model(inputs=inp, outputs=output)
model.summary()

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_indices, Y_train_oh, epochs = 100, batch_size = 32, shuffle=True)
##validation_split=0.2, 
##initial_epoch=0

In [0]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Accuracy Teste = ", acc)

In [0]:
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Correto emoji:'+ label_to_emoji(Y_test[i]) + ' previsão: '+ X_test[i] + label_to_emoji(num).strip())

In [0]:
x_test = np.array(['not feeling happy'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

#  Modelo LSTM

In [0]:
# Definir o modelo "model"
import keras.backend as K
from keras.layers import Lambda
hidden_size=128

inp = Input(shape=(maxLen,))
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
embedding = embedding_layer(inp)
x = LSTM(hidden_size, return_sequences=False)(embedding)
#x = Flatten()(embedding)
#x = Dense(128)(x)
x = Dropout(0.5)(x)
logits = Dense(5)(x)
output = Activation('softmax')(logits)

model = Model(inputs=inp, outputs=output)

In [0]:
#Chamar o modelo
model.summary()

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)

model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)
##validation_split=0.2, 
##initial_epoch=0

In [0]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Accuracy Teste = ", acc)

In [0]:
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Correto emoji:'+ label_to_emoji(Y_test[i]) + ' previsão: '+ X_test[i] + label_to_emoji(num).strip())

In [0]:
x_test = np.array(['feeling happy'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))